# Downloading an NWB File
In order to analyze some data, you'll need to have some data. The [DANDI Archive](https://dandiarchive.org/) is used to store [NWB](https://www.nwb.org/) files in datasets called **dandisets**. Typically an NWB file contains the data for just one experimental session, while a dandiset contains all the related data files yielded from a project. This notebook allows you to download from public dandisets or private dandisets (called **embargoed** dandisets) via the [DANDI Python API](https://dandi.readthedocs.io/en/latest/modref/index.html). To download embargoed dandisets from DANDI, you will need to make an account on the DANDI Archive and must be given access by the owner of the dandiset.

### Environment Setup

In [1]:
### if running on Google Colab, run this cell once, then restart the runtime and run the rest of the notebook
import os
if "COLAB_GPU" in os.environ:
    !git clone https://github.com/AllenInstitute/openscope_databook.git
    %cd openscope_databook
    %pip install -e .

In [2]:
from dandi import dandiapi

### Download Configuration
Here you can configure the download. Browse the DANDI Archive for a dandiset you're interested in and use its ID in `dandiset_id`. Also set `download_loc` to the relative filepath of the directory you'd like to download to. If you're accessing an embargoed dandiset, you should set `authenticate` to True, and set `dandi_api_key` to your DANDI API Key, which can be found if you click on your profile icon in the top-right corner on the DANDI Archive website.

In [3]:

dandiset_id = "000021" #the set ID can be found under the title of the dandiset
download_loc = "." #a filepath specifies the location of a file relative to the current directory (not starting from the root directory)
#explain how to access the filepath?
authenticate = False
dandi_api_key = ""

In [4]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)

print(f"Got dandiset {my_dandiset}")

Got dandiset DANDI:000021/draft


### Downloading Just One File
Set `dandi_filepath` to the path of the file you want to download within the dandiset. You can get this by navigating to the file you want to download on the DANDI Archive website and pressing on the `i` icon. There, you can copy the filepath from the field labeled `path`. Don't include a leading `/`.

In [5]:
dandi_filepath = "sub-699733573/sub-699733573_ses-715093703.nwb"

In [6]:
filename = dandi_filepath.split("/")[-1]
filepath = f"{download_loc}/{filename}"

print(filename)
print(filepath)


sub-699733573_ses-715093703.nwb
./sub-699733573_ses-715093703.nwb


In [7]:
file = my_dandiset.get_asset_by_path(dandi_filepath)
# this may take awhile, especially if the file to download is large
file.download(filepath)

print(f"Downloaded file to {filepath}")


PermissionError: [Errno 13] Permission denied

### Downloading Entire Dandiset
If you'd like to do a lot of work with the files in a dandiset, you might want to download the entire thing or some portion of the dandiset. Be prepared, though; This could take a significant amount of space on your drive and a significant amount of time. If you want to just download all the files within a directory of the dandiset, you can set the first argument of `download_directory` below to a more specific path within the dandiset.

In [ ]:
# patience isn't just a virtue, it's a requirement
my_dandiset.download_directory("./", f"{download_loc}/{dandiset_id}")

print(f"Downloaded directory to {download_loc}/{dandiset_id}")

Downloaded directory to ./000021
